### Chat bot simple avec mémoire

<img src="../assets/simple_chatbot_with_memory.png" alt="drawing" width="600"/>

#### Variables d'environnement
Charger les variables d'environnement (clé OpenAI, etc.)

In [11]:


from dotenv import load_dotenv
load_dotenv()

True

#### Initialisation du Chat Model
Le client du LLM est instancié. La classe `ChatOpenAI` implémente la classe de base `BaseChatModel` partagée avec `AzureChatOpenAi`, `ChatMistralAI`, ...

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

#### Preparation du Prompt Template

Le `prompt template` contient 3 parties:
* Un message `System` contenant les instructions pour le LLM (rôle, formattage, etc.)
* Un `Placeholder` dans lequel sera injecté le contenu de la mémoire
* Un message `Human` dans lequel la question utilisateur sera injectée

In [12]:
from langchain_core.prompts import (
    ChatPromptTemplate,  
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate, 
    MessagesPlaceholder
)

prompt_template = ChatPromptTemplate.from_messages(
    [
       SystemMessagePromptTemplate.from_template( "You are a helpful AI bot. Respond in french."),
       MessagesPlaceholder(variable_name="history"),
       HumanMessagePromptTemplate.from_template("{question}")
    ]
)

print(prompt_template)

input_variables=['history', 'question'] input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful AI bot. Respond in french.')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]


#### Création de la chaîne finale
La chaîne est une simple séquence de deux `Runnables`:
* Le *prompt template*, dans lequel sera injectée les *inputs*
* Le *model* prenant en entrée le *prompt* variabilisé et dont l'output sera la *réponse finale*

In [4]:
chain = prompt_template | model

#### Invocation de la chaîne
La chaîne est invoquée en lui fournissant les inputs nécessaires:
* La question utilisateur
* La liste des messages précédents issue de l'objet mémoire, pour le moment un tableau vide

In [14]:
first_question = "Hi ! My name is Laurent"

first_response = chain.invoke(
    {
        "question": first_question,
        "history": []
    }
)
print(f"Réponse:\n  {first_response.content}")

Réponse:
  Bonjour Laurent ! Comment puis-je vous aider aujourd'hui ?


#### Gestion de la mémoire

Les LLM étant stateless, sans gestion de la mémoire de notre part toute question suivante posée sera décorellée de la précédente. 

In [15]:
second_question="What is my name ?"

response_without_memory = chain.invoke(
    {
        "question": second_question,
        "history": []
    }
)
response_without_memory.content

"Je suis désolé, mais je ne peux pas connaître ton nom. Comment t'appelles-tu ?"

Ici pour gérer la mémoire nous utiliserons la classe `ChatMessageHistory` qui va stocker la mémoire dans un simple array. Pour une vrai utilisation, on préférera évidemment persister la mémoire. Pour cela Langchain propose de nombreuses intégrations: `SQL`, `Postgres`, `Mongo`, `ElasticSearch`, `Redis`,... (qui implémentent toutes la même interface)

In [9]:
from langchain.memory import ChatMessageHistory

chat_history = ChatMessageHistory()
chat_history.add_user_message(first_question)
chat_history.add_ai_message(first_response.content)

Lors de la nouvelle invocation, il nous suffit de passer en input la liste des messages de la mémoire

In [16]:
response_with_memory = chain.invoke(
    {
        "question": "What is my name ?",
        "history": chat_history.messages
    }
)
response_with_memory.content

'Votre nom est Laurent.'